In [2]:
import pandas as pd
import requests as rq
import json
from jenkinsapi.jenkins import Jenkins

In [7]:
#j = Jenkins("http://localhost.com", "admin", "p")
print(Jenkins)

<class 'jenkinsapi.jenkins.Jenkins'>


In [2]:
symbols = ['ABBV', 'ABC', 'ALL', 'BC', 'BERY', 'BIO', 'BMY', 'CIEN', 'CMC', 'DKS', 'EQH', 'HUM', 'INT', 'LAD', 'LEN', 'LRCX', 'NOK', 'STAG', 'TPX', 'UNH', 'VIAC', 'WPC']

# Fundamentals

In [3]:
stocks_fundamentals = []
for s in symbols:
    res = rq.get("https://eodhistoricaldata.com/api/fundamentals/"+s+"?api_token=&fmt=csv")
    if res.status_code != 200:
        print(f'Unable to find {s}')
        continue
        
    res = json.loads(res.content)
    stocks_fundamentals.append(res)
    

In [4]:
for s in stocks_fundamentals:
    print(s['General']['GicSector'])

Health Care
Health Care
Financials
Consumer Discretionary
Materials
Health Care
Health Care
Information Technology
Materials
Consumer Discretionary
Financials
Health Care
Energy
Consumer Discretionary
Consumer Discretionary
Information Technology
Information Technology
Real Estate
Consumer Discretionary
Health Care
Communication Services
Real Estate


# Get pricing target

In [11]:
price_targets = []
for s in symbols:
    symbol = s
    signal = None
    max_price_target = None
    avg_price_target = None
    min_price_target = None
    last_updated = None

    req = rq.get(f'https://bank.tipranks.com/api/stocks/getData/{symbol}/?benchmark=1&period=1')
    if req.status_code != 200:
        print(f'Unable to get info for {symbol}')
        continue
        
    info = req.json()
    
    price_target_counter = 0
    best_analyst_rating = 0
    for e in info['experts']:
        pt = e['ratings'][0]['priceTarget']
        if pt is None:
            continue
            
        # avg price target
        price_target_counter += 1
        if avg_price_target is None:
            avg_price_target = pt
        else:
            avg_price_target += pt
            
        # min price target
        if min_price_target is None:
            min_price_target = pt
        else:
            if pt < min_price_target:
                min_price_target = pt
        
        # max price target
        if max_price_target is None:
            max_price_target = pt
        else:
            if pt > max_price_target:
                max_price_target = pt
    
    
    avg_price_target = avg_price_target / price_target_counter
    current_price = info['prices'][-1]['p']
    
    result = {'symbol': symbol, 'max_price_target': max_price_target, 'avg_price_target': avg_price_target, 'min_price_target': min_price_target, 'current_price': current_price}
    price_targets.append(result)
    print(result)
print(price_targets)

{'symbol': 'ABBV', 'max_price_target': 127.0, 'avg_price_target': 109.0, 'min_price_target': 95.0, 'current_price': 94.17}
{'symbol': 'ABC', 'max_price_target': 127.0, 'avg_price_target': 114.2, 'min_price_target': 98.0, 'current_price': 97.55}
{'symbol': 'ALL', 'max_price_target': 135.0, 'avg_price_target': 122.4, 'min_price_target': 101.0, 'current_price': 94.3}
{'symbol': 'BC', 'max_price_target': 88.0, 'avg_price_target': 68.75, 'min_price_target': 37.0, 'current_price': 62.1}
{'symbol': 'BERY', 'max_price_target': 65.0, 'avg_price_target': 60.666666666666664, 'min_price_target': 55.0, 'current_price': 52.48}
{'symbol': 'BIO', 'max_price_target': 625.0, 'avg_price_target': 565.0, 'min_price_target': 430.0, 'current_price': 498.84}
{'symbol': 'BMY', 'max_price_target': 80.0, 'avg_price_target': 68.9, 'min_price_target': 50.0, 'current_price': 62.37}
{'symbol': 'CIEN', 'max_price_target': 73.0, 'avg_price_target': 59.666666666666664, 'min_price_target': 31.0, 'current_price': 56.03}


## Merge with fundamentals

In [8]:
for s in price_targets:
    for f in stocks_fundamentals:
        if s['symbol'] == f['General']['Code']:
            s['GicSector'] = f['General']['GicSector']

In [9]:
df = pd.DataFrame().from_dict(price_targets)
print(df)

   symbol  max_price_target  avg_price_target  min_price_target  \
0    ABBV             127.0        109.000000              95.0   
1     ABC             127.0        114.200000              98.0   
2     ALL             135.0        122.400000             101.0   
3      BC              88.0         68.750000              37.0   
4    BERY              65.0         60.666667              55.0   
5     BIO             625.0        565.000000             430.0   
6     BMY              80.0         68.900000              50.0   
7    CIEN              73.0         59.666667              31.0   
8     CMC              25.0         21.375000              17.0   
9     DKS              71.0         57.000000              27.0   
10    EQH              29.0         25.000000              21.0   
11    HUM             510.0        438.318182             350.0   
12    INT              36.0         30.000000              24.0   
13    LAD             280.0        212.250000             110.

In [10]:
df.to_csv('seeking_alpha_analysis.csv')